# Using soleimanian/financial-roberta-large-sentiment fine tuned BERT model for sentiment analysis on TOP Stock and Crypto Picks by Hanryck Brar

# Loading BERT Model For Sentiment Analysis

## Installing The Dependencies

In [1]:
%pip install transformers
%pip install AutoTokenizer
%pip install AutoModelForSequenceClassification
%pip install newspaper3k

Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement AutoTokenizer (from versions: none)
ERROR: No matching distribution found for AutoTokenizer
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement AutoModelForSequenceClassification (from versions: none)
ERROR: No matching distribution found for AutoModelForSequenceClassification
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Importing The Necessary Libraries

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
from transformers import AutoTokenizer
from transformers import pipeline
import pandas as pd
import nltk
nltk.download('punkt')
import newspaper
from newspaper import Article


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>


## Loading Bert Model and Tokenizier

In [3]:
#We used this BERT model: https://huggingface.co/soleimanian/financial-roberta-large-sentiment trained on financial data
sentiment_analysis = pipeline("sentiment-analysis",model="soleimanian/financial-roberta-large-sentiment")

## Testing The Model

In [4]:
result=sentiment_analysis("Tesla bullish signs")
sentiment=result[0]['label']
print(sentiment)

positive


# Actual Sentiment Analysis Bot using Bert,NEWS API and Newspaper3k

## The stocks and crypto we want to get sentiment analysis on

In [5]:
import pandas as pd
#The top 100 cryptocurries from Coinmarketcap
file_path = 'BestCryptos.csv'
bestcryptodata = pd.read_csv(file_path)
# The top 30 stocks from Dow jones 30
file_path = 'Best30Stocks.csv'
beststockdata = pd.read_csv(file_path)

## The lists in which we will store the data sentiment bot predicts

In [6]:
Name=[]
Symbol=[]
Postive_Score=[]
Negative_Score=[]
Postive_titles=[]
Negative_titles=[]
Postive_descriptions=[]
Negative_descriptions=[]
Postive_Articles=[]
Negative_Articles=[]
Overal_Sentiment=[]

Name1=[]
Symbol1=[]
Postive_Score1=[]
Negative_Score1=[]
Postive_titles1=[]
Negative_titles1=[]
Postive_descriptions1=[]
Negative_descriptions1=[]
Postive_Articles1=[]
Negative_Articles1=[]
Overal_Sentiment1=[]

## Built Some Useful Functions

In [7]:
postive = 0
negative = 0
postive2 = 0
negative2 = 0
postive3 = 0
negative3 = 0

# Assigns Numeric value based sentiment given from bert model
def sentimentnumeric(label):
  if str(label) == 'negative':
    return 1
  elif str(label) == 'positive':
    return 5
  elif str(label) == 'neutral':
    return 3

def titlescheck(titlesgiven):
  global postive, negative
  result=sentiment_analysis(titlesgiven)
  sentimentlabel=result[0]['label']
  sentiment=sentimentnumeric(sentimentlabel)
  if sentiment>3:
    postive=postive+1
  elif sentiment<3:
    negative=negative+1

def descriptionscheck(descriptionsgiven):
  global postive2, negative2
  result=sentiment_analysis(descriptionsgiven)
  sentimentlabel=result[0]['label']
  sentiment=sentimentnumeric(sentimentlabel)
  if sentiment>3:
    postive2=postive2+1
  elif sentiment<3:
    negative2=negative2+1

#The Basics of downloading the article to memory
def articlesearch(urlused):
  try:
    global postive3, negative3
    #The reason for this is because biztoc website doesn't scrap properly
    if "biztoc" in urlused:
      return "biztoc doesn't work"
    else:
      article = Article(str(urlused))
      article.download()
      article.parse()
      article.nlp()
      summary = article.summary
      result = sentiment_analysis(summary)
      sentimentlabel = result[0]['label']
      sentiment = sentimentnumeric(sentimentlabel)
      if sentiment > 3:
          postive3 = postive3 + 1
      elif sentiment < 3:
          negative3 = negative3 + 1
  except Exception as e:
        return 'not viable'
#refer not global but local variables
def sentiment_score(postive,postive2,postive3,negative,negative2,negative3):
  tp=postive2+postive+postive3
  tn=negative2+negative+negative3
  if tp>tn:
    return 'Positive'
  elif tn>tp:
    return 'Negative'
  elif tn==tp:
    return 'Neutral'

def postive_scorecalculator(postive,postive2,postive3):
  tp=postive+postive2+postive3
  return int(tp)

def negative_scorecalculator(negative,negative2,negative3):
  tn=negative+negative2+negative3
  return int(tn)

## Pulls Top 15 articles using News API and Newpaper3k using articlesearch() functiongathers summary on those 30 articles and passes that summary to the sentiment analysis bot

In [8]:
import requests

def get_related_news(query):
    API_KEY = ""  # Replace with your NewsAPI key
    url = f"https://newsapi.org/v2/everything"
    params = {
        "q": query,            # Query for articles containing "btc"
        "apiKey": API_KEY,

    }
    #does http request to api server to get back data in json file
    response = requests.get(url, params=params)

    if response.status_code == 200:
      #Using this method the json file is converted to dictionary
        data = response.json()
        #This allows to find all articles related to query if nothing is return
        #automatically return a '[]'
        articles = data.get("articles")
        return articles
    else:
        print("Failed to retrieve news articles.")
        return []

def the_sentiment_bot(value,name,typeitem):
  if typeitem=="cryptos":
    query =f'{value} future price'
  if typeitem=="stocks":
    query =f'{value} price'
  max_articles=15

#This boiler plate code that is necessary but I don't understand it.
  if __name__ == "__main__":
      articles = get_related_news(query)


      print(f'Current:{name}')
      for index, article in enumerate(articles, start=1):
          title = article.get("title", "N/A")
          source = article.get("source", {}).get("name", "N/A")
          description = article.get("description", "N/A")
          url = article.get("url", "#")
          #descriptionscheck(str(description))
          titlescheck(str(title))
          articlesearch(str(url))
          if index > max_articles:
              break
          #print(f"Article {index}:\n")
          #print(f"Title: {str(title)}")
          #print(f"Source: {source}")
          #print(f"Description: {str(description)}")
          #print(f"URL: {url}\n")
      print(f'Overal Sentiment:{sentiment_score(postive,postive2,postive3,negative,negative2,negative3)}')

      #This is to seperate data based on stocks and cryptos in data frame
      if typeitem =="stocks":
        Name.append(name)
        Symbol.append(value)
        Postive_Score.append(postive_scorecalculator(postive,postive2,postive3)-negative_scorecalculator(negative,negative2,negative3))
        Negative_Score.append(negative_scorecalculator(negative,negative2,negative3)-postive_scorecalculator(postive,postive2,postive3))
        Postive_titles.append(postive)
        Negative_titles.append(negative)
        Postive_descriptions.append(postive2)
        Negative_descriptions.append(negative2)
        Postive_Articles.append(postive3)
        Negative_Articles.append(negative3)
        Overal_Sentiment.append(sentiment_score(postive,postive2,postive3,negative,negative2,negative3))
      if typeitem == "cryptos":
        Name1.append(name)
        Symbol1.append(value)
        Postive_Score1.append(postive_scorecalculator(postive,postive2,postive3)-negative_scorecalculator(negative,negative2,negative3))
        Negative_Score1.append(negative_scorecalculator(negative,negative2,negative3)-postive_scorecalculator(postive,postive2,postive3))
        Postive_titles1.append(postive)
        Negative_titles1.append(negative)
        Postive_descriptions1.append(postive2)
        Negative_descriptions1.append(negative2)
        Postive_Articles1.append(postive3)
        Negative_Articles1.append(negative3)
        Overal_Sentiment1.append(sentiment_score(postive,postive2,postive3,negative,negative2,negative3))


## Sentiment Analysis on TOP Stocks

In [10]:
symbols=['XOM']
names=["ExxonMobil"]
for i,j in zip(symbols,names):
    the_sentiment_bot(i,j,"stocks")
    postive = 0
    negative = 0
    postive2 = 0
    negative2 = 0
    postive3 = 0
    negative3 = 0


Current:ExxonMobil
Overal Sentiment:Negative
2


In [11]:
start_index=0
for i in range(1):
  start_index=i*3
  for index, row in beststockdata.iloc[start_index:].iterrows():
    symbol = row['Symbol']
    name = row['Name']
    the_sentiment_bot(symbol, name,"stocks")
    postive = 0
    negative = 0
    postive2 = 0
    negative2 = 0
    postive3 = 0
    negative3 = 0
    if index == start_index + 2:
      print("First Batch Done")
      break


Current:American Express Co
Overal Sentiment:Positive
Current:Amgen Inc
Overal Sentiment:Positive
Current:Apple Inc
Overal Sentiment:Positive
First Batch Done


## Creating Way So It Is Easy To Visualize The Top Stocks

In [12]:
topsentimentstocks = {
        'Name': Name,
        'Symbol': Symbol,
        'Positive Score':Postive_Score,
        'Negative Score': Negative_Score,
        '#Positive Titles On Articles':Postive_titles ,
        '#Negative Titles On Articles': Negative_titles,
        '#Positive Descriptions On Articles': Postive_descriptions,
        '#Negative Descriptions On Articles': Negative_descriptions,
        '#Positive summaries about Articles':  Postive_Articles,
        '#Negative summaries about Articles':  Negative_Articles,
        'Overal Sentiment': Overal_Sentiment,
    }

In [13]:
topsentimentstocks = pd.DataFrame(topsentimentstocks)
topsentimentstocks.sort_values(by="Positive Score",ascending=False)


,Name,Symbol,Positive Score,Negative Score,#Positive Titles On Articles,#Negative Titles On Articles,#Positive Descriptions On Articles,#Negative Descriptions On Articles,#Positive summaries about Articles,#Negative summaries about Articles,Overal Sentiment
2,American Express Co,AXP,8,-8,8,0,0,0,0,0,Positive
3,Amgen Inc,AMGN,3,-3,6,3,0,0,0,0,Positive
4,Apple Inc,AAPL,2,-2,6,4,0,0,0,0,Positive
0,ExxonMobil,XOM,-1,1,2,3,0,0,0,0,Negative
1,ExxonMobil,XOM,-1,1,2,3,0,0,0,0,Negative


## For Cryptos

In [14]:
for i in range(1):
  start_index = 3*i
  for index,row in bestcryptodata.iloc[start_index:].iterrows():
    symbol = row['Symbol']
    name = row['Name']
    the_sentiment_bot(symbol, name,"cryptos")
    if index == start_index + 2:
        print("First Batch Done")
        postive = 0
        negative = 0
        postive2 = 0
        negative2 = 0
        postive3 = 0
        negative3 = 0
        break

Current:Bitcoin
Overal Sentiment:Positive
Current:Ethereum
Overal Sentiment:Positive
Current:Binance Coin
Overal Sentiment:Positive
First Batch Done


## Creating Way So It Is Easy To Visualize The Top Cryptos

In [15]:
topsentimentcrypto = {
        'Name': Name1,
        'Symbol': Symbol1,
        'Positive Score':Postive_Score1,
        'Negative Score': Negative_Score1,
        '#Positive Titles On Articles':Postive_titles1 ,
        '#Negative Titles On Articles': Negative_titles1,
        '#Positive Descriptions On Articles': Postive_descriptions1,
        '#Negative Descriptions On Articles': Negative_descriptions1,
        '#Positive summaries about Articles':  Postive_Articles1,
        '#Negative summaries about Articles':  Negative_Articles1,
        'Overal Sentiment': Overal_Sentiment1,
    }

In [16]:
topsentimentcrypto = pd.DataFrame(topsentimentcrypto)
topsentimentcrypto.sort_values(by="Positive Score",ascending=True)

,Name,Symbol,Positive Score,Negative Score,#Positive Titles On Articles,#Negative Titles On Articles,#Positive Descriptions On Articles,#Negative Descriptions On Articles,#Positive summaries about Articles,#Negative summaries about Articles,Overal Sentiment
0,Bitcoin,BTC,6,-6,8,2,0,0,0,0,Positive
1,Ethereum,ETH,7,-7,13,6,0,0,0,0,Positive
2,Binance Coin,BNB,17,-17,25,8,0,0,0,0,Positive
